## Boulder Watershed data accumulation:

This is a snippet to collect the ICEsat-2 data from the Boulder Watershed, processed to a variety of length scales, and to save them to local files.  

In [ ]:
import sys
import logging
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy
import time
from sliderule import icesat2
#import pointCollection as pc

In [ ]:
# Configure Logging
icesat2_logger = logging.getLogger("sliderule.icesat2")
icesat2_logger.setLevel(logging.CRITICAL)
# Create Console Output
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
icesat2_logger.addHandler(ch)

In [ ]:
# Configure ICESat-2 API
icesat2.init("icesat2sliderule.org", False)
# Configure Region of Interest
region = [ {"lon":-105.82971551223244, "lat": 39.81983728534918},
           {"lon":-105.30742121965137, "lat": 39.81983728534918},
           {"lon":-105.30742121965137, "lat": 40.164048017973755},
           {"lon":-105.82971551223244, "lat": 40.164048017973755},
           {"lon":-105.82971551223244, "lat": 39.81983728534918} ]

In [ ]:
# note that this can be run with 'srt':icesat2.SRT_LAND to use the land classification

for L in [5, 10, 20, 40 ,80]:
    for conf in [icesat2.CNF_SURFACE_LOW, icesat2.CNF_SURFACE_MEDIUM, icesat2.CNF_SURFACE_HIGH]:
        # Build ATL06 Request
        parms = {
            "poly": region,
            "srt": icesat2.SRT_LAND_ICE,
            "cnf": conf,
            "ats": L/4,
            "cnt": 10,
            "len": L,
            "res": 20.0,
            "maxi": 10
        }
        out_file='Boulder_ice_cnf%d_L%d.h5' %(conf, L)
        print(out_file)
        # Request ATL06 Data
        rsps = icesat2.atl06p(parms, "atlas-s3")
        # save the output:
        pc.data().from_dict({field:np.array(rsps[field]) for field in rsps.keys() if field != '__rectype'}).to_h5(out_file)
        